# Phase 2: Story Structure

This phase is the most important on this project. We are making the story structure so we basically have 3 steps for this:

* Start defining the ASP approach.
* Use the Sentiment Analysis to create emotional integration
* The system generates multiple valid outlines that satisfy our constrains, ensuring both logical coherence and emotional resonance.

So in more detail what we will be doing in code is defining multiple functions, here are the key components:

1. Scene structure foundation.
2. Narrative Functions.
3. Constrain System.
4. Implementation features.
 

In [10]:
# import the principal libraries
import pandas as pd 
import numpy as np 
import torch
import os
from groq import Groq, Client
from dotenv import load_dotenv 
load_dotenv()

import dspy


# import typos
from typing import Dict, Tuple, Optional, List, Set, Optional

# import other libraries
from enum import Enum 
from dataclasses import dataclass

In [11]:
open_ai_api_key = os.getenv("OPENAI_API_KEY")

In [39]:
# import keys from .env
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
client = Client(
    api_key=groq_api_key
    )
llmo = dspy.LM('openai/gpt-4o-mini', api_key=open_ai_api_key)
dspy.configure(lm=llmo)
# Load the clean document
# doc_path = "cleaned_text.txt"
# with open(doc_path, "r") as f:
#     INPUT_FILE = f.read()

In [41]:
test_file = """El águila, la liebre y el escarabajo.

Estaba una liebre siendo perseguida por un águila, y viéndose perdida pidió ayuda a un escarabajo, suplicándole que le ayudara.

Le pidió el escarabajo al águila que perdonara a su amiga. Pero eláguila, despreciando la insignificancia del escarabajo, devoró a la liebre en su presencia.

Desde entonces, buscando vengarse, el escarabajo observaba los lugares donde el águila ponía sus huevos, y haciéndolos rodar, los tiraba a tierra. Viéndose el águila echada del lugar a donde quiera que fuera, recurrió a Zeus pidiéndole un lugar seguro para depositar sus huevos.

Le ofreció Zeus colocarlos en su regazo, pero el escarabajo, viendo la táctica escapatoria, hizo una bolita de estiércol, voló y la dejó caer sobre el regazo de Zeus.
Se levantó entonces Zeus para sacudirse aquella suciedad, y tiró por tierra los huevos sin darse cuenta. Por eso desde entonces, las águilas no ponen huevos en la época en que salen a volar los escarabajos.

Enseñanza: Nunca desprecies lo que parece insignificante, pues no hay ser tan débil que no pueda alcanzarte."""

### 1. Structure Scene Fundation

In [42]:
class ExtractRelevantInformation(dspy.Signature):
    """Extract the relevant information and key themes about this story"""
    input_text: str = dspy.InputField()
    reasoning: str = dspy.OutputField(desc="Step by step reasoning and extraction of key themes")
    moral_of_the_story: str = dspy.OutputField(desc="The moral lesson or teaching (Enseñanza) of the story")
    key_themes: str = dspy.OutputField(desc="Key themes and main characters identified in the story")

class StoryCreation(dspy.Signature):
    """Based on the key themes and moral of the story, create an entertaining story"""
    input_text: str = dspy.InputField()
    themes: str = dspy.InputField()
    moral: str = dspy.InputField()
    story: str = dspy.OutputField(desc="Create an entertaining story incorporating the themes and moral")


In [43]:
class StoryStructure(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract_relevant_information = dspy.ChainOfThought(ExtractRelevantInformation)
        self.story_creation = dspy.ChainOfThought(StoryCreation)

    def forward(self, input_text):
        # First, extract themes and moral from the initial text
        extracted_info = self.extract_relevant_information(
            input_text=input_text
        )
        
        # Then create the story using the extracted information
        story = self.story_creation(
            input_text=input_text,
            themes=extracted_info.key_themes,
            moral=extracted_info.moral_of_the_story
        )
        
        return {
            'themes': extracted_info.key_themes,
            'moral': extracted_info.moral_of_the_story,
            'story': story.story,
            'reasoning': extracted_info.reasoning
        }

In [44]:
# Initialize the story generator
generator = StoryStructure()

# Generate the story and return all components
result = generator(test_file)

In [46]:
def print_story_result(result_dict):
    print("\n" + "="*50 + " TEMAS " + "="*50)
    print(result_dict['themes'])
    
    print("\n" + "="*50 + " MORAL " + "="*50)
    print(result_dict['moral'])
    
    print("\n" + "="*50 + " HISTORIA " + "="*50)
    print(result_dict['story'])
    
    print("\n" + "="*50 + " RAZONAMIENTO " + "="*50)
    print(result_dict['reasoning'])
    print("\n" + "="*120)


In [52]:
# Print the result
print_story_result(result)


================================================== TEMAS ==================================================
Los temas clave incluyen la venganza, la subestimación de los demás, y la astucia. Los personajes principales son el águila, que representa la fuerza y el desprecio, la liebre, que simboliza la vulnerabilidad, y el escarabajo, que encarna la inteligencia y la perseverancia.

================================================== MORAL ==================================================
Nunca desprecies lo que parece insignificante, pues no hay ser tan débil que no pueda alcanzarte.

================================================== HISTORIA ==================================================
En un vasto y hermoso bosque, donde los árboles susurraban secretos y los ríos cantaban melodías, vivía un majestuoso águila. Con sus alas extendidas, dominaba el cielo, y todos los animales la temían. Un día, mientras volaba en busca de su próxima presa, avistó a una liebre que saltaba alegremen

In [51]:
with open("new_story.txt", 'w', encoding='utf-8') as f:
        f.write(result["story"])